In [1]:
import pandas as pd
from fancyimpute import KNN
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

C:\Users\rupal\Anaconda3\envs\PythonData_3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [5]:
#combine train and test data into one dataframe
data = pd.concat([train, test], ignore_index=True)

In [6]:
print(data.shape, train.shape, test.shape)

(14204, 12) (8523, 12) (5681, 11)


### Data Exploring

In [7]:
print(data.isnull().sum())

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  4016
Outlet_Type                     0
dtype: int64


17% of the items have missing values for weight

In [8]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 12 columns):
Item_Fat_Content             14204 non-null object
Item_Identifier              14204 non-null object
Item_MRP                     14204 non-null float64
Item_Outlet_Sales            8523 non-null float64
Item_Type                    14204 non-null object
Item_Visibility              14204 non-null float64
Item_Weight                  11765 non-null float64
Outlet_Establishment_Year    14204 non-null int64
Outlet_Identifier            14204 non-null object
Outlet_Location_Type         14204 non-null object
Outlet_Size                  10188 non-null object
Outlet_Type                  14204 non-null object
dtypes: float64(4), int64(1), object(7)
memory usage: 1.3+ MB
None


In [9]:
data.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000
mean,141.004977,2181.288914,0.065953,12.792854,1997.830681
std,62.086938,1706.499616,0.051459,4.652502,8.371664
min,31.290000,33.290000,0.000000,4.555000,1985.000000
25%,94.012000,834.247400,0.027036,8.710000,1987.000000
50%,142.247000,1794.331000,0.054021,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


Minimum item visibility is 0. Perhaps these items are placed on higher shelves or far behind other items and not visible. 

In [10]:
data[data.Item_Visibility == 0.0].count()

Item_Fat_Content             879
Item_Identifier              879
Item_MRP                     879
Item_Outlet_Sales            526
Item_Type                    879
Item_Visibility              879
Item_Weight                  733
Outlet_Establishment_Year    879
Outlet_Identifier            879
Outlet_Location_Type         879
Outlet_Size                  649
Outlet_Type                  879
dtype: int64

6% of the items have 0 visibility.  

In [11]:
uniq_data = [data[col].unique() for col in data]
print
uniq_data

[array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object),
 array(['FDA15', 'DRC01', 'FDN15', ..., 'NCF55', 'NCW30', 'NCW05'],
       dtype=object),
 array([249.8092,  48.2692, 141.618 , ...,  72.8696,  82.125 , 188.053 ]),
 array([3735.138 ,  443.4228, 2097.27  , ..., 3608.636 , 2778.3834,
              nan]),
 array(['Dairy', 'Soft Drinks', 'Meat', 'Fruits and Vegetables',
        'Household', 'Baking Goods', 'Snack Foods', 'Frozen Foods',
        'Breakfast', 'Health and Hygiene', 'Hard Drinks', 'Canned',
        'Breads', 'Starchy Foods', 'Others', 'Seafood'], dtype=object),
 array([0.0160473 , 0.01927822, 0.01676007, ..., 0.1429909 , 0.07352856,
        0.10472015]),
 array([ 9.3  ,  5.92 , 17.5  , 19.2  ,  8.93 , 10.395, 13.65 ,    nan,
        16.2  , 11.8  , 18.5  , 15.1  , 17.6  , 16.35 ,  9.   , 13.35 ,
        18.85 , 14.6  , 13.85 , 13.   ,  7.645, 11.65 ,  5.925, 19.25 ,
        18.6  , 18.7  , 17.85 , 10.   ,  8.85 ,  9.8  , 13.6  , 21.35 ,
        12.15 ,  6.4

### Data Wrangling

In [12]:
# Combine Item_Fat_Content Categories 1.LF, low, fat, and Low Fat into Low Fat and 2. reg and Regular into Regular

data.Item_Fat_Content = data.Item_Fat_Content.replace({'LF': 'Low Fat',
                                   'low fat': 'Low Fat',
                                   'reg': 'Regular'
                                   })
print(data.Item_Fat_Content.value_counts())                                  

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64


In [13]:
# Find the size of the different outlets 

df = data.groupby(['Outlet_Type','Outlet_Size']).sum()
df 

Item_MRP  Item_Outlet_Sales  \
Outlet_Type       Outlet_Size                                   
Grocery Store     Small        123862.1000       1.796941e+05   
Supermarket Type1 High         219172.4492       2.142664e+06   
                  Medium       218802.9588       2.183970e+06   
                  Small        436990.2746       4.386518e+06   
Supermarket Type2 Medium       217987.3906       1.851823e+06   
Supermarket Type3 Medium       219838.2488       3.453926e+06   

                               Item_Visibility  Item_Weight  \
Outlet_Type       Outlet_Size                                 
Grocery Store     Small              94.502641          NaN   
Supermarket Type1 High               93.555174    19859.980   
                  Medium             91.450099    19844.655   
                  Small             187.566136    39707.560   
Supermarket Type2 Medium             92.723425    19794.425   
Supermarket Type3 Medium             94.075671          NaN   

                               Outlet_Establishment_Year  
Outlet_Type       Outlet_Size                             
Grocery Store     Small                          1746800  
Supermarket Type1 High                           3085811  
                  Medium                         3098450  
                  Small                          6201550  
Supermarket Type2 Medium                         3105914  
Supermarket Type3 Medium                         3094615

All Grocery stores are small, Supermarket Type2 and Type3 are medium, Supermarket Type1 are small and medium.

In [14]:
# Find the common value for Supermarket Type1 and use it as the Supermarket Type1 size

s = data[data.Outlet_Type =='Supermarket Type1']
print(s.Outlet_Size.mode())

0    Small
dtype: object


In [15]:
# Replace missing values in the Outlet Size corresponding to the Outlet Type

dict = {'Grocery Store':'Small', 'Supermarket Type1': 'Small', 'Supermarket Type2': 'Medium', 'Supermarket Type3': 'Medium'}
data['Outlet_Size'] = data['Outlet_Type'].fillna(data['Outlet_Size']).map(dict)
data.head()

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,Small,Supermarket Type1
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,Medium,Supermarket Type2
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,Small,Supermarket Type1
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.000000,19.20,1998,OUT010,Tier 3,Small,Grocery Store
4,Low Fat,NCD19,53.8614,994.7052,Household,0.000000,8.93,1987,OUT013,Tier 3,Small,Supermarket Type1


In [16]:
print(data.Outlet_Size.value_counts())

Small     11099
Medium     3105
Name: Outlet_Size, dtype: int64


In [17]:
# Which items have missing values for weight?

d = data[data['Item_Weight'].isnull()]
print(d['Item_Type'].value_counts())

Fruits and Vegetables    346
Snack Foods              336
Household                263
Frozen Foods             251
Dairy                    195
Baking Goods             186
Canned                   186
Health and Hygiene       148
Meat                     129
Soft Drinks              123
Breads                    73
Hard Drinks               59
Others                    52
Starchy Foods             42
Breakfast                 34
Seafood                   16
Name: Item_Type, dtype: int64


In [18]:
# Impute the missing values for Item_Weight with the KNN method from from the fancyimpute package from sklearn. 
# The KNN method imputes missing values based on the weight of other similar items.

df1 = data.groupby(['Item_Type', 'Item_Fat_Content']).apply(lambda x: 
                        pd.DataFrame(KNN(k=3).fit_transform(x.loc[:, ['Item_MRP', 
                    'Item_Outlet_Sales', 'Item_Visibility', 'Item_Weight' ]]), 
                                                      columns = ['Item_MRP', 
                    'Item_Outlet_Sales', 'Item_Visibility', 'Item_Weight' ] )).reset_index()

Imputing row 1/542 with 1 missing, elapsed time: 0.080
Imputing row 101/542 with 0 missing, elapsed time: 0.081
Imputing row 201/542 with 0 missing, elapsed time: 0.082
Imputing row 301/542 with 0 missing, elapsed time: 0.082
Imputing row 401/542 with 1 missing, elapsed time: 0.084
Imputing row 501/542 with 1 missing, elapsed time: 0.086
Imputing row 1/542 with 1 missing, elapsed time: 0.042
Imputing row 101/542 with 0 missing, elapsed time: 0.043
Imputing row 201/542 with 0 missing, elapsed time: 0.043
Imputing row 301/542 with 0 missing, elapsed time: 0.044
Imputing row 401/542 with 1 missing, elapsed time: 0.046
Imputing row 501/542 with 1 missing, elapsed time: 0.048
Imputing row 1/544 with 0 missing, elapsed time: 0.041
Imputing row 101/544 with 0 missing, elapsed time: 0.042
Imputing row 201/544 with 0 missing, elapsed time: 0.042
Imputing row 301/544 with 0 missing, elapsed time: 0.043
Imputing row 401/544 with 2 missing, elapsed time: 0.045
Imputing row 501/544 with 1 missing, 

Imputing row 1/112 with 0 missing, elapsed time: 0.003
Imputing row 101/112 with 1 missing, elapsed time: 0.003


In [19]:
# Update the dataframe 'data' with the imputed missing values for Item_Weight

data.update(df1.Item_Weight)

In [20]:
data.shape

(14204, 12)

In [21]:
# Add a new feature Outlet_Age

data['Outlet_Age'] = 2019 - data['Outlet_Establishment_Year']

In [22]:
# Categorize the Item_Type into Food, Drink and Consumable. The first two variables of the Item_Identifier FD-Food DR-Drink, 
# NC-NonConsumable can be used as the categories. Create a new feature with these categories.

data['Item_Category'] = data['Item_Identifier'].str[:2]

data.head()

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type,Outlet_Age,Item_Category
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,12.279885,1999,OUT049,Tier 1,Small,Supermarket Type1,20,FD
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,13.567337,2009,OUT018,Tier 3,Medium,Supermarket Type2,10,DR
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,11.100000,1999,OUT049,Tier 1,Small,Supermarket Type1,20,FD
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.000000,10.300034,1998,OUT010,Tier 3,Small,Grocery Store,21,FD
4,Low Fat,NCD19,53.8614,994.7052,Household,0.000000,9.100000,1987,OUT013,Tier 3,Small,Supermarket Type1,32,NC


In [23]:
# Split the dataset into train and test, and delete Item_Outlet_Sales from the test dataset.

train_df = data.iloc[:8523, :]
test_df = data.iloc[8523:, :]
test_df = test_df.drop('Item_Outlet_Sales', axis=1)

In [24]:
print(train_df.shape, test_df.shape)

(8523, 14) (5681, 13)


In [25]:
# Save the cleaned train and test datasets as csv files. 

train_df.to_csv('data/train_clean.csv', index=False)
test_df.to_csv('data/test_clean.csv', index=False)